In [1]:
# main libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.nn import functional as F
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms, datasets

In [29]:
from torch.utils.tensorboard import SummaryWriter

In [19]:
# custom libraries
root_main = os.getcwd()
os.chdir("..")
import TorchCommon as TC
os.chdir(root_main)

In [40]:
# main pyperparametrs
valid_size=0.2
epochs=1
nrm_mean=0.5
nrm_std=0.5
num_workers=0
root_ds = "D:\GitHub\pytorch-lab\Dataset" # dataset root
root_bm = "D:\GitHub\pytorch-lab\Best_Models" # best models root

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [42]:
# transform
trans=transforms.Compose([
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((nrm_mean,) , (nrm_std,))
])

In [43]:
#load dataset
train_data=datasets.MNIST(root=root_ds,
                          train=True, transform=trans, download=True)

test_data=datasets.MNIST(root=root_ds,
                          train=False, transform=trans, download=True)

In [44]:
valid_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=valid_sampler)
test_loader =DataLoader(test_data,  batch_size= batch_size, num_workers=num_workers, shuffle=True)

In [45]:
#sampler
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [46]:
# Network
class Net(nn.Module):
    # Assign Intrinsic Properties of Your Neural Network
    def __init__(self):
        super().__init__()
        # Weights of Layer 1th and Layer 2th Are Intrinsic Properties
        self.fc1 = nn.Linear(784, 256, bias=True)
        self.fc2 = nn.Linear(256, 100, bias=True)
        self.fc3 = nn.Linear(100, 10, bias=True)

    # Wiring of Your Network
    def feed_forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = F.relu_(x)
        x = self.fc2(x)
        x = F.relu_(x)
        x = self.fc3(x)
        return x

# Hyperparameters Searching

In [49]:
#loss_valid_min=np.Inf
for batch_size in [2,64,1024]:
    for lr in [0.1, 0.01, 0.001, 0.0001]:
        train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)
        #create model and set loss function and optimizer
        model=Net().to(device)
        criterion =nn.CrossEntropyLoss()
        optimizer=optim.SGD(model.parameters(), lr=lr)
        root_tb=f'runs/batch {batch_size} LR {lr}'
        writer = SummaryWriter(root_tb)

        for epoch in range(1,epochs+1):
            loss_train, acc_train = TC.train(model,train_loader, device, optimizer, criterion, epoch)
            #loss_valid, acc_valid = TC.valid(model, valid_loader, device, criterion)
            #loss_valid_min = TC.save_model(model, optimizer, epoch, root_bm, loss_valid_min, loss_valid)

            writer.add_scalar("train loss", scalar_value= loss_train, global_step=epoch)
            writer.add_scalar("train accuracy", scalar_value= acc_train, global_step=epoch)
            writer.add_hparams({'batch_size' : batch_size, 'LR' : lr} , {'Loss' : loss_train, 'Accuracy' : acc_train})

############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 2.25	Acc_Train : 0.14
############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 0.34	Acc_Train : 0.89
############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 0.61	Acc_Train : 0.83
############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 1.93	Acc_Train : 0.47
############################################################
###### Epoch 1 #############################################
##########

# Visualizing Dataset Images and Network Weights

In [ ]:
#loss_valid_min=np.Inf
for batch_size in [2]:
    for lr in [0.1]:
        train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)
        #create model and set loss function and optimizer
        model=Net().to(device)
        criterion =nn.CrossEntropyLoss()
        optimizer=optim.SGD(model.parameters(), lr=lr)
        root_tb=f'runs/batch {batch_size} LR {lr}'
        writer = SummaryWriter(root_tb)

        for epoch in range(1,epochs+1):
            loss_train, acc_train = TC.train(model,train_loader, device, optimizer, criterion, epoch)
            #loss_valid, acc_valid = TC.valid(model, valid_loader, device, criterion)
            #loss_valid_min = TC.save_model(model, optimizer, epoch, root_bm, loss_valid_min, loss_valid)

            writer.add_scalar("train loss", scalar_value= loss_train, global_step=epoch)
            writer.add_scalar("train accuracy", scalar_value= acc_train, global_step=epoch)
            writer.add_hparams({'batch_size' : batch_size, 'LR' : lr} , {'Loss' : loss_train, 'Accuracy' : acc_train})